In [1]:
import polars as pl

DATA_PATH = "E:/30-39 Estudos/36 Mestrado/36.03 Deep learning para computação visual/Projeto/flickr_scraper/images"
METADATA = DATA_PATH + "/metadata.csv"

In [ ]:
df = pl.read_csv(METADATA)
filepaths = DATA_PATH + "/" + df["year"].cast(pl.String) + "/" + df["id"].cast(pl.String) + ".jpg"
filepaths.head()

In [ ]:
import filetype as ftp

def is_gif(filename: str) -> bool:
    guess = ftp.guess(filename)
    return guess.mime == "image/gif"

filepaths = DATA_PATH + "/" + df["year"].cast(pl.String) + "/" + df["id"].cast(pl.String) + ".jpg"
filepaths = filepaths.to_frame(name="filepath")
#is_gif_df = filepaths["filepath"].map_elements(is_gif, return_dtype=pl.Boolean)
is_gif_df = filepaths.select(pl.col("filepath").map_batches(is_gif))
is_gif_df

In [ ]:
new_df = pl.concat((filepaths, is_gif_df.to_frame(name="is_gif")), how="horizontal")
new_df

In [ ]:
new_df.filter(pl.col("is_gif"))

In [ ]:
#def is_gif_col(df: pl.DataFrame) -> pl.DataFrame:
df = pl.read_csv(METADATA)
df = df.with_columns(
    (
        DATA_PATH + "/" + pl.col("year").cast(pl.String) + "/" + pl.col("id").cast(pl.String) + ".jpg"
    ).alias("filepath")
)
df = df.with_columns(
    (
        df["filepath"].map_elements(is_gif)
    ).alias("is_gif")
)
df
#df.filter(~pl.col("is_gif"))

In [ ]:
def filter_out_gifs(df: pl.DataFrame) -> pl.DataFrame:
    original_columns = df.columns
    df = df.with_columns(
    (
        DATA_PATH + "/" + pl.col("year").cast(pl.String) + "/" + pl.col("id").cast(pl.String) + ".jpg"
    ).alias("filepath")
    )
    df = df.with_columns(
        (
            df["filepath"].map_elements(is_gif, return_dtype=pl.Boolean)
        ).alias("is_gif")
    )
    df = df.filter(~pl.col("is_gif"))
    return df.select(original_columns)

df = pl.read_csv(METADATA)
df = filter_out_gifs(df)
df